In [ ]:
import glob
from tqdm import tqdm
from subprocess import PIPE, Popen
import pandas as pd
from joblib import Parallel, delayed
import multiprocessing
import os
import yaml

In [ ]:
with open("gnomad_db/gnomad_columns.yaml") as f:
    columns = yaml.load(f, Loader=yaml.FullLoader)
columns = columns["base_columns"] + columns["value_columns"]
print(len(columns))
columns[:10]

In [ ]:
gnomad_vcf_location = "test"
tables_location = "test"

In [ ]:
# get gnomAD files
files = glob.glob(f"{gnomad_vcf_location}/*.bgz")
print(len(files))
files

In [ ]:
# write gnomAD files to these tables:
tables_location = [f'{tables_location}/{file.split("/")[-1].replace(".vcf.bgz", "")}.tsv.gz' for file in files]
tables_location

In [ ]:
cpu_count = int(multiprocessing.cpu_count())
cpu_count

In [ ]:
# extract needed columns
# if running DIRECTLY from notebook, add module load i12g/bcftools; in the beginning of cmd
def create_table(file, table_location):
    query_string = "%" + "\t%".join(columns) + "\n"
    
    if not os.path.exists(table_location):
        cmd = f"bcftools query -f '{query_string}' {file} | gzip > {table_location}"
        p = Popen(cmd, shell=True, stdout=PIPE, stderr=PIPE)
        print(p.communicate())
    


In [ ]:
# run bcftools in parallel
Parallel(cpu_count)(delayed(create_table)(file, table_location) for file, table_location in tqdm(zip(files, tables_location)))